In [2]:
from pyspark import SparkConf, SparkContext

In [4]:
#conf = SparkConf().setMaster("yarn")
sc = SparkContext()

In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [8]:
text = sc.textFile("hdfs://192.168.1.100:9000/csv/student.csv")

In [9]:
first = text.first()
first

'學校名稱,日間∕進修別,等級別,一年級男生,一年級女生,二年級男生,二年級女生,三年級男生,三年級女生,四年級男生,四年級女生,五年級男生,五年級女生,六年級男生,六年級女生,七年級男生,七年級女生,延修生男生,延修生女生,縣市名稱,學年度'

In [10]:
data = text.filter(lambda x: x != first)
data.first()

'國立政治大學,日,博士,117,76,79,62,94,58,98,57,75,53,61,43,59,41,0,0,臺北市,103'

In [11]:
data = data.map(lambda x: x.split(","))

In [12]:
data.take(1)

[['國立政治大學',
  '日',
  '博士',
  '117',
  '76',
  '79',
  '62',
  '94',
  '58',
  '98',
  '57',
  '75',
  '53',
  '61',
  '43',
  '59',
  '41',
  '0',
  '0',
  '臺北市',
  '103']]

In [13]:
data.map(lambda x: x[0:3] + x[3:5] + x[19:]).take(3)

[['國立政治大學', '日', '博士', '117', '76', '臺北市', '103'],
 ['國立政治大學', '日', '碩士', '626', '707', '臺北市', '103'],
 ['國立政治大學', '日', '學士', '859', '1359', '臺北市', '103']]

In [13]:
first

'學校名稱,日間∕進修別,等級別,一年級男生,一年級女生,二年級男生,二年級女生,三年級男生,三年級女生,四年級男生,四年級女生,五年級男生,五年級女生,六年級男生,六年級女生,七年級男生,七年級女生,延修生男生,延修生女生,縣市名稱,學年度'

In [14]:
def to_int(x):
    for i, j in enumerate(x[3:-2]):
        x[i + 3] = int(j)
    return x

In [15]:
def student_filter(school=None, div=None, level=None, grade=None, gender=None, loc=None, year=None):
    filter_data = data
    
    if school:
        filter_data = filter_data.filter(lambda x: x[0] == school)
    if div:
        filter_data = filter_data.filter(lambda x: x[1] == div)
    if level:
        filter_data = filter_data.filter(lambda x: x[2] == level)   
    if loc:
        filter_data = filter_data.filter(lambda x: x[19] == loc)
    if year:
        filter_data = filter_data.filter(lambda x: x[20] == year)
    if grade:
        if grade == "一年級":
            filter_data = filter_data.map(lambda x: x[0:5] + x[19:])
        elif grade == "二年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[5:7] + x[19:])
        elif grade == "三年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[7:9] + x[19:])
        elif grade == "四年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[9:11] + x[19:])
        elif grade == "五年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[11:13] + x[19:])
        elif grade == "六年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[13:15] + x[19:])
        elif grade == "七年級":
            filter_data = filter_data.map(lambda x: x[0:3] + x[15:17] + x[19:])
        elif grade == "延修生":
            filter_data = filter_data.map(lambda x: x[0:3] + x[17:])
        if gender == "男生":
            filter_data = filter_data.map(lambda x: x[0:4] + x[5:])
        elif gender == "女生":
            filter_data = filter_data.map(lambda x: x[0:3] + x[4:])
    if grade == None and gender:
        if gender == "男生":
            filter_data = filter_data.map(lambda x: x[0:3] + x[3:18:2] + x[19:])
        elif gender == "女生":
            filter_data = filter_data.map(lambda x: x[0:3] + x[4:19:2] + x[19:])
    
    filter_data = filter_data.map(to_int)
    return filter_data.map(lambda x: x[0:3] + [sum(x[3:-2])] + x[-2:]).collect()
    

In [15]:
def initializaion():
    textRDD = sc.textFile("hdfs://name:9000/csv/student.csv")
    text1 = textRDD.collect()
    textList = []
    for data1 in text1:
        textList.append(data1.split(','))
    tableRDD = sc.parallelize(textList)
    
    return tableRDD

In [17]:
student_filter(school='國立臺灣大學', gender='男生')

[['國立臺灣大學', '日', '博士', 2984, '臺北市', '103'],
 ['國立臺灣大學', '日', '碩士', 5940, '臺北市', '103'],
 ['國立臺灣大學', '日', '學士', 9425, '臺北市', '103'],
 ['國立臺灣大學', '職', '碩士', 771, '臺北市', '103'],
 ['國立臺灣大學', '進', '學士', 4, '臺北市', '103'],
 ['國立臺灣大學', '日', '博士', 2794, '臺北市', '104'],
 ['國立臺灣大學', '日', '碩士', 6136, '臺北市', '104'],
 ['國立臺灣大學', '日', '學士', 9441, '臺北市', '104'],
 ['國立臺灣大學', '職', '碩士', 778, '臺北市', '104'],
 ['國立臺灣大學', '進', '學士', 1, '臺北市', '104'],
 ['國立臺灣大學', '日', '博士', 2679, '臺北市', '105'],
 ['國立臺灣大學', '日', '碩士', 6160, '臺北市', '105'],
 ['國立臺灣大學', '日', '學士', 9403, '臺北市', '105'],
 ['國立臺灣大學', '職', '碩士', 842, '臺北市', '105'],
 ['國立臺灣大學', '日', '博士', 2542, '臺北市', '106'],
 ['國立臺灣大學', '日', '碩士', 6188, '臺北市', '106'],
 ['國立臺灣大學', '日', '學士', 9393, '臺北市', '106'],
 ['國立臺灣大學', '職', '碩士', 930, '臺北市', '106'],
 ['國立臺灣大學', '進', '學士', 1, '臺北市', '106'],
 ['國立臺灣大學', '日', '博士', 2405, '臺北市', '107'],
 ['國立臺灣大學', '日', '碩士', 6178, '臺北市', '107'],
 ['國立臺灣大學', '日', '學士', 9447, '臺北市', '107'],
 ['國立臺灣大學', '職', '碩士', 904, '臺北市', '107']]

In [30]:
student_filter(school='國立臺灣大學', grade='一年級', gender='男生')

[['國立臺灣大學', '日', '博士', 489, '臺北市', '103'],
 ['國立臺灣大學', '日', '碩士', 2429, '臺北市', '103'],
 ['國立臺灣大學', '日', '學士', 2028, '臺北市', '103'],
 ['國立臺灣大學', '職', '碩士', 252, '臺北市', '103'],
 ['國立臺灣大學', '進', '學士', 0, '臺北市', '103'],
 ['國立臺灣大學', '日', '博士', 456, '臺北市', '104'],
 ['國立臺灣大學', '日', '碩士', 2457, '臺北市', '104'],
 ['國立臺灣大學', '日', '學士', 2075, '臺北市', '104'],
 ['國立臺灣大學', '職', '碩士', 259, '臺北市', '104'],
 ['國立臺灣大學', '進', '學士', 0, '臺北市', '104'],
 ['國立臺灣大學', '日', '博士', 471, '臺北市', '105'],
 ['國立臺灣大學', '日', '碩士', 2478, '臺北市', '105'],
 ['國立臺灣大學', '日', '學士', 2108, '臺北市', '105'],
 ['國立臺灣大學', '職', '碩士', 288, '臺北市', '105'],
 ['國立臺灣大學', '日', '博士', 413, '臺北市', '106'],
 ['國立臺灣大學', '日', '碩士', 2483, '臺北市', '106'],
 ['國立臺灣大學', '日', '學士', 2096, '臺北市', '106'],
 ['國立臺灣大學', '職', '碩士', 345, '臺北市', '106'],
 ['國立臺灣大學', '進', '學士', 0, '臺北市', '106'],
 ['國立臺灣大學', '日', '博士', 408, '臺北市', '107'],
 ['國立臺灣大學', '日', '碩士', 2445, '臺北市', '107'],
 ['國立臺灣大學', '日', '學士', 2126, '臺北市', '107'],
 ['國立臺灣大學', '職', '碩士', 321, '臺北市', '107']]